# 🎯 YOLO Segmentation Anti-Cheat Model Fine-tuning

**IMPORTANT**: This notebook is for training **YOLO Segmentation models** (yolo11s-seg, yolo11x-seg, etc.).

This notebook will fine-tune the existing YOLO segmentation model to improve detection of:
- 📱 **Phone** (currently weak - max ~0.2%)
- 📄 **Material/Paper** (currently weak - max ~0.1%)
- 👤 **Person** (needs improvement - max ~5%)
- 🎧 **Headphones** (already good - max ~44%)

## Instructions:
1. Upload your existing `best.pt` segmentation model to Google Drive
2. Run all cells in order
3. Download the new `best.onnx` file when done
4. Replace `Intelligence-Test/public/models/anticheat_yolo11s.onnx`

## Note on Dataset Format:
This notebook automatically converts bounding box labels to segmentation polygon format
so that detection datasets can be used to train segmentation models.

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install dependencies
!pip install ultralytics -q
!pip install onnx onnxruntime -q

import os
import shutil
import yaml
import numpy as np
from pathlib import Path
from ultralytics import YOLO

print("✅ Dependencies installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.4 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Dependencies installed!


In [3]:
# Configuration
TARGET_CLASSES = ['person', 'phone', 'material', 'headphones']

# Dataset URLs from Roboflow (phone and material focused)
DATASETS = [
    # Phone datasets (HIGH PRIORITY - model is weak here)
    ("phone_1", "https://app.roboflow.com/ds/5ReObgnLbQ?key=HTPSgVzDLW"),
    ("phone_2", "https://app.roboflow.com/ds/f9k54F7Azq?key=eYssUekSYc"),

    # Paper/Material datasets (HIGH PRIORITY)
    ("paper_1", "https://app.roboflow.com/ds/inuabMtp6t?key=jbu7HTlrBf"),
    ("paper_2", "https://app.roboflow.com/ds/b4oxAhlW40?key=4A761Kjm5F"),

    # Headphones (for balance)
    ("headphones_1", "https://app.roboflow.com/ds/qqqEeSKAlk?key=GT1Xa65onI"),
    ("headphones_2", "https://app.roboflow.com/ds/cKHwOqmuda?key=qL10KsWlBt"),

    # Person
    ("person_1", "https://app.roboflow.com/ds/PwRwV0c1jL?key=FgXbXeqlpH"),
]

# Class mapping
CLASS_MAPPING = {
    'person': 'person', 'student': 'person', 'face': 'person', 'head': 'person',
    'human': 'person', 'people': 'person', 'man': 'person', 'woman': 'person',
    'phone': 'phone', 'mobile': 'phone', 'cell phone': 'phone',
    'telephone': 'phone', 'smartphone': 'phone', 'cellphone': 'phone',
    'mobile phone': 'phone', 'iphone': 'phone', 'android': 'phone',
    'ProductRecog - v2 2024-11-05 7:03am': 'phone',
    'paper': 'material', 'document': 'material', 'book': 'material',
    'notebook': 'material', 'notes': 'material', 'sheet': 'material',
    'material': 'material', 'cheat sheet': 'material', 'PAPER': 'material', 'Paper': 'material',
    'headphone': 'headphones', 'headphones': 'headphones',
    'earphone': 'headphones', 'earphones': 'headphones',
    'headset': 'headphones', 'earbuds': 'headphones', 'earbud': 'headphones',
    'airpods': 'headphones', 'ear device': 'headphones', 'Headphone': 'headphones',
    'left earbud': 'headphones', 'eardevice': 'headphones',
}

def normalize_class(class_name):
    class_name = class_name.lower().strip()
    for key, target in CLASS_MAPPING.items():
        if key.lower() == class_name:
            return TARGET_CLASSES.index(target)
    return -1

def bbox_to_segment(bbox_coords):
    """
    Convert YOLO bounding box (x_center, y_center, width, height) to
    segmentation polygon format (x1 y1 x2 y2 x3 y3 x4 y4).

    This is required for training segmentation models with detection datasets.
    Returns None if the input is invalid or results in out-of-bounds coordinates.
    """
    try:
        xc, yc, w, h = map(float, bbox_coords)
        # Validate input values are in valid range [0, 1]
        if not (0 <= xc <= 1 and 0 <= yc <= 1 and 0 < w <= 1 and 0 < h <= 1):
            return None
        # Calculate corner points (normalized coordinates)
        x1, y1 = xc - w/2, yc - h/2  # Top-left
        x2, y2 = xc + w/2, yc - h/2  # Top-right
        x3, y3 = xc + w/2, yc + h/2  # Bottom-right
        x4, y4 = xc - w/2, yc + h/2  # Bottom-left
        # Clamp values to [0, 1] range
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(1, x2), max(0, y2)
        x3, y3 = min(1, x3), min(1, y3)
        x4, y4 = max(0, x4), min(1, y4)
        return f"{x1} {y1} {x2} {y2} {x3} {y3} {x4} {y4}"
    except (ValueError, TypeError):
        return None

print("✅ Configuration loaded!")
print(f"Target classes: {TARGET_CLASSES}")
print(f"\n📌 Note: Labels will be converted to segmentation format (polygon coordinates)")

✅ Configuration loaded!
Target classes: ['person', 'phone', 'material', 'headphones']

📌 Note: Labels will be converted to segmentation format (polygon coordinates)


In [4]:
# Download datasets
!mkdir -p /content/raw_datasets
%cd /content/raw_datasets

for name, url in DATASETS:
    if not os.path.exists(name):
        print(f"📥 Downloading {name}...")
        !mkdir -p {name}
        !curl -L "{url}" > {name}/dataset.zip 2>/dev/null
        !unzip -q {name}/dataset.zip -d {name}
        !rm {name}/dataset.zip
    else:
        print(f"✓ {name} already exists")

print("\n✅ All datasets downloaded!")

/content/raw_datasets
📥 Downloading phone_1...
📥 Downloading phone_2...
📥 Downloading paper_1...
📥 Downloading paper_2...
📥 Downloading headphones_1...
📥 Downloading headphones_2...
📥 Downloading person_1...

✅ All datasets downloaded!


In [5]:
# Merge and convert datasets to SEGMENTATION format
OUTPUT_DIR = '/content/merged_dataset'

# Clear and create output directory
!rm -rf {OUTPUT_DIR}
os.makedirs(f"{OUTPUT_DIR}/train/images", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/train/labels", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/valid/images", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/valid/labels", exist_ok=True)

total_train = 0
total_valid = 0
bbox_converted = 0  # Count of bounding boxes converted to segments
seg_preserved = 0   # Count of segmentation labels preserved

for name, _ in DATASETS:
    dataset_dir = f"/content/raw_datasets/{name}"
    if not os.path.exists(dataset_dir):
        continue

    # Find data.yaml
    data_yaml = None
    for root, dirs, files in os.walk(dataset_dir):
        if 'data.yaml' in files:
            data_yaml = os.path.join(root, 'data.yaml')
            break

    if not data_yaml:
        print(f"⚠️ No data.yaml in {name}")
        continue

    with open(data_yaml, 'r') as f:
        config = yaml.safe_load(f)

    source_classes = config.get('names', [])
    if isinstance(source_classes, dict):
        source_classes = list(source_classes.values())

    print(f"\n📂 Processing {name}...")
    print(f"   Source classes: {source_classes}")

    for split in ['train', 'valid', 'test']:
        img_dir = None
        lbl_dir = None

        # Try different directory structures
        for try_path in [dataset_dir, os.path.dirname(data_yaml)]:
            if os.path.exists(os.path.join(try_path, split, 'images')):
                img_dir = os.path.join(try_path, split, 'images')
                lbl_dir = os.path.join(try_path, split, 'labels')
                break

        if not img_dir or not os.path.exists(img_dir):
            continue

        out_split = 'train' if split in ['train', 'test'] else 'valid'
        count = 0

        for img_file in os.listdir(img_dir):
            if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue

            # Copy image
            src_img = os.path.join(img_dir, img_file)
            dst_img = os.path.join(OUTPUT_DIR, out_split, 'images', f"{name}_{img_file}")
            shutil.copy(src_img, dst_img)

            # Convert label to SEGMENTATION format
            lbl_file = os.path.splitext(img_file)[0] + '.txt'
            src_lbl = os.path.join(lbl_dir, lbl_file)
            dst_lbl = os.path.join(OUTPUT_DIR, out_split, 'labels', f"{name}_{lbl_file}")

            if os.path.exists(src_lbl):
                with open(src_lbl, 'r') as f:
                    lines = f.readlines()

                new_lines = []
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) < 5:
                        continue

                    old_class_id = int(parts[0])
                    if old_class_id < len(source_classes):
                        old_class_name = source_classes[old_class_id]
                        new_class_id = normalize_class(old_class_name)

                        if new_class_id >= 0:
                            # Check if this is bounding box (5 parts) or segmentation (9+ parts)
                            # Bounding box: class x_center y_center width height (5 values)
                            # Segmentation: class x1 y1 x2 y2 x3 y3 x4 y4... (9+ values for rectangle/polygon)
                            if len(parts) == 5:
                                # Bounding box format: class x_center y_center width height
                                # Convert to segmentation polygon format
                                segment_coords = bbox_to_segment(parts[1:])
                                if segment_coords:
                                    new_lines.append(f"{new_class_id} {segment_coords}")
                                    bbox_converted += 1
                            elif len(parts) >= 9:
                                # Already segmentation format: class x1 y1 x2 y2 x3 y3 x4 y4 ...
                                new_lines.append(f"{new_class_id} {' '.join(parts[1:])}")
                                seg_preserved += 1
                            # Skip labels with 6-8 parts as they're likely malformed

                if new_lines:
                    with open(dst_lbl, 'w') as f:
                        f.write('\n'.join(new_lines))
                    count += 1

        if out_split == 'train':
            total_train += count
        else:
            total_valid += count

        if count > 0:
            print(f"   {split} -> {out_split}: {count} images")

# Create data.yaml
data_yaml_content = {
    'path': OUTPUT_DIR,
    'train': 'train/images',
    'val': 'valid/images',
    'names': {i: name for i, name in enumerate(TARGET_CLASSES)},
    'nc': len(TARGET_CLASSES),
}

with open(f"{OUTPUT_DIR}/data.yaml", 'w') as f:
    yaml.dump(data_yaml_content, f, default_flow_style=False)

print(f"\n" + "="*50)
print(f"✅ Dataset prepared for SEGMENTATION training!")
print(f"   Train images: {total_train}")
print(f"   Valid images: {total_valid}")
print(f"   Classes: {TARGET_CLASSES}")
print(f"\n📊 Label conversion:")
print(f"   Bounding boxes converted to polygons: {bbox_converted}")
print(f"   Segmentation labels preserved: {seg_preserved}")
print(f"="*50)


📂 Processing phone_1...
   Source classes: ['ProductRecog - v2 2024-11-05 7-03am', 'phone']
   train -> train: 103 images
   valid -> valid: 29 images
   test -> train: 15 images

📂 Processing phone_2...
   Source classes: ['6', 'phone', 'undefined']
   train -> train: 1216 images
   valid -> valid: 266 images
   test -> train: 181 images

📂 Processing paper_1...
   Source classes: ['document']
   train -> train: 3459 images
   valid -> valid: 453 images
   test -> train: 1174 images

📂 Processing paper_2...
   Source classes: ['paper']
   train -> train: 52 images
   valid -> valid: 16 images
   test -> train: 7 images

📂 Processing headphones_1...
   Source classes: ['earphone']
   train -> train: 82 images
   valid -> valid: 14 images
   test -> train: 8 images

📂 Processing headphones_2...
   Source classes: ['earphone']
   train -> train: 362 images
   valid -> valid: 105 images
   test -> train: 44 images

📂 Processing person_1...
   Source classes: ['book']
   train -> train: 1

In [6]:
# Upload or specify base model path
# IMPORTANT: Use a SEGMENTATION model (yolo11s-seg.pt, yolo11x-seg.pt, etc.)

# Option 1: Upload from local machine
# from google.colab import files
# uploaded = files.upload()  # Upload best.pt
# BASE_MODEL_PATH = list(uploaded.keys())[0]

# Option 2: Use from Google Drive (recommended) - your existing segmentation model
BASE_MODEL_PATH = '/content/drive/MyDrive/Intelligence-Test-Models/anticheat_objects_v2_headphones/weights/best.pt'

# Option 3: Start from pretrained YOLO11 SEGMENTATION model
# BASE_MODEL_PATH = 'yolo11s-seg.pt'  # Note: use -seg variant!

print(f"Base model: {BASE_MODEL_PATH}")
if os.path.exists(BASE_MODEL_PATH):
    print("✅ Model file found!")
    # Verify it's a segmentation model
    if '-seg' in BASE_MODEL_PATH.lower() or 'seg' in BASE_MODEL_PATH.lower():
        print("✅ Confirmed: This appears to be a segmentation model")
else:
    if BASE_MODEL_PATH.startswith('yolo'):
        print(f"📥 Will download pretrained model: {BASE_MODEL_PATH}")
    else:
        print("❌ Model file not found! Please check path.")
        print("\n💡 TIP: For segmentation, use models like:")
        print("   - yolo11s-seg.pt (small, fast)")
        print("   - yolo11m-seg.pt (medium)")
        print("   - yolo11x-seg.pt (large, accurate)")

Base model: /content/drive/MyDrive/Intelligence-Test-Models/anticheat_objects_v2_headphones/weights/best.pt
✅ Model file found!


In [7]:
# Fine-tune the SEGMENTATION model
model = YOLO(BASE_MODEL_PATH)

print("🚀 Starting fine-tuning SEGMENTATION model...")
print("This may take 30-60 minutes depending on dataset size.")

results = model.train(
    data=f"{OUTPUT_DIR}/data.yaml",
    epochs=50,              # Number of epochs
    imgsz=640,              # Image size
    batch=16,               # Batch size (reduce if OOM)
    patience=15,            # Early stopping
    lr0=0.001,              # Lower LR for fine-tuning
    lrf=0.01,               # Final LR factor
    warmup_epochs=3,        # Warmup
    freeze=10,              # Freeze first 10 layers
    project='/content/drive/MyDrive/Intelligence-Test-Models',
    name='anticheat_finetuned_seg_v3',
    exist_ok=True,
    device=0,               # GPU
    verbose=True,
)

print("\n✅ Training completed!")

🚀 Starting fine-tuning SEGMENTATION model...
This may take 30-60 minutes depending on dataset size.
Ultralytics 8.3.237 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/merged_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/Intelligence-Test-Models/anticheat_objects_v2_headphones/weights/best.pt, momen

In [8]:
# Export to ONNX
BEST_MODEL_PATH = '/content/drive/MyDrive/Intelligence-Test-Models/anticheat_finetuned_seg_v3/weights/best.pt'

model = YOLO(BEST_MODEL_PATH)

print("📦 Exporting segmentation model to ONNX...")
model.export(
    format='onnx',
    imgsz=640,
    simplify=True,
    dynamic=False,
    opset=17
)

onnx_path = BEST_MODEL_PATH.replace('.pt', '.onnx')
print(f"\n✅ ONNX model saved to: {onnx_path}")

📦 Exporting segmentation model to ONNX...
Ultralytics 8.3.237 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLO11s-seg summary (fused): 113 layers, 10,068,364 parameters, 0 gradients, 32.8 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/Intelligence-Test-Models/anticheat_finetuned_seg_v3/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 40, 8400), (1, 32, 160, 160)) (19.6 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<=1.19.1', 'onnxslim>=0.1.71', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 14 packages in 183ms
Prepared 4 packages in 8.12s
Uninstalled 1 package in 981ms
Installed 4 packages in 440ms
 + colorama==0.4.6
 - onnx==1.20.0
 + onnx==1.19.1
 + onnxruntime-gpu==1.23.2
 + onnxslim==0.1.80

requirements: AutoU

In [9]:
# Test the new model
import onnxruntime as ort
from PIL import Image, ImageDraw

session = ort.InferenceSession(onnx_path)
input_name = session.get_inputs()[0].name
classes = ['person', 'phone', 'material', 'headphones']

def test_image(name, img):
    img = img.convert('RGB').resize((640, 640))
    img_array = np.array(img).astype(np.float32) / 255.0
    img_array = np.transpose(img_array, (2, 0, 1))
    img_array = np.expand_dims(img_array, axis=0)

    outputs = session.run(None, {input_name: img_array})
    # Segmentation models output: (detection_output, mask_output)
    output = outputs[0]
    class_scores = output[0, 4:8, :]

    print(f"\n{name}:")
    for i, cls in enumerate(classes):
        max_score = class_scores[i].max()
        print(f"  {cls}: max={max_score:.4f}")

# Test with phone simulation
phone_img = Image.new('RGB', (640, 640), color=(200, 200, 200))
draw = ImageDraw.Draw(phone_img)
draw.rectangle([250, 200, 390, 450], fill=(20, 20, 20))
draw.rectangle([260, 210, 380, 440], fill=(50, 50, 80))
test_image("Phone simulation", phone_img)

# Test with headphones simulation
headphones_img = Image.new('RGB', (640, 640), color=(200, 200, 200))
draw = ImageDraw.Draw(headphones_img)
draw.arc([200, 150, 440, 350], 180, 0, fill=(30, 30, 30), width=20)
draw.ellipse([180, 280, 260, 400], fill=(40, 40, 40))
draw.ellipse([380, 280, 460, 400], fill=(40, 40, 40))
test_image("Headphones simulation", headphones_img)

print("\n" + "="*50)
print("If phone scores improved (>0.1), the fine-tuning worked!")
print("="*50)


Phone simulation:
  person: max=0.0000
  phone: max=0.0228
  material: max=0.0001
  headphones: max=0.0001

Headphones simulation:
  person: max=0.0000
  phone: max=0.0003
  material: max=0.0000
  headphones: max=0.0006

If phone scores improved (>0.1), the fine-tuning worked!


In [10]:
# Download the ONNX file
print("\n📋 NEXT STEPS:")
print("1. Download the ONNX file from Google Drive")
print(f"   Location: {onnx_path}")
print("2. Rename it to: anticheat_yolo11s.onnx")
print("3. Copy to: Intelligence-Test/public/models/")
print("4. Rebuild and deploy the web app")
print("\n🎉 Done!")


📋 NEXT STEPS:
1. Download the ONNX file from Google Drive
   Location: /content/drive/MyDrive/Intelligence-Test-Models/anticheat_finetuned_seg_v3/weights/best.onnx
2. Rename it to: anticheat_yolo11s.onnx
3. Copy to: Intelligence-Test/public/models/
4. Rebuild and deploy the web app

🎉 Done!
